In [35]:
from utils.checkpoints import load_checkpoint
from constants import *
from models.densenet121 import DenseNet121_change_avg
from dataloader.dataloader import HmDataset
from dataloader.transforms import transforms
from main.fit import fit

from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# get test data loader
train_dataset = HmDataset(df_path='./dataset/train.csv', transforms=transforms, mode=DATASET_MODE)
valid_dataset = HmDataset(df_path='./dataset/valid.csv', transforms=transforms, mode=DATASET_MODE)
test_dataset = HmDataset(df_path='./dataset/test.csv', transforms=transforms, mode=DATASET_MODE)

train_loader = DataLoader(train_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)
test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False,
                         drop_last=False,
                         num_workers=4)

# create model
model = DenseNet121_change_avg()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=0.9)

# load model
model =  DenseNet121_change_avg()
model, optimizer, _, epoch = load_checkpoint('./checkpoints/200601_223249_DenseNet121_LR0.001_BS64_BCELoss/  8.pth', model, optimizer)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

[Test]	Loss:0.09219: 100%|██████████| 333/333 [00:12<00:00, 26.18it/s]


In [59]:
from tqdm import tqdm
import pandas as pd
def get_features(data_loader):
    feature_array = []
    
    for filename, target, inputs in tqdm(data_loader, position=0):

        target, inputs = target.to(device), inputs.to(device)
        pred_label, features = model(inputs)
        
        row = [filename[0], pred_label.squeeze().cpu().detach().numpy(), features.squeeze().cpu().detach().numpy()]
        feature_array.append(row)
        
    df = pd.DataFrame(feature_array, columns=['filename','label','features'])
    return df

In [60]:
df = get_features(test_loader)
df.to_csv('./dataset/valid_features.csv')

100%|██████████| 333/333 [00:13<00:00, 24.76it/s]
